# Obtención de Dataset

Pasos:
1. Colocar 4 arauco en las esquinas.
2. Medir y calibrar la cámara.
3. Detectar el robot con un 5to arauco distinto.
4. Detectar muros/contornos con threshold.

# Prueba de detección de Aruco Marker

In [1]:
import cv2 as cv
import numpy as np

# Cámara:
cap = cv.VideoCapture(0)

# Diccionarios y detectores Aruco Markers:
aruco_dict = cv.aruco.getPredefinedDictionary(cv.aruco.DICT_4X4_100)
parameters = cv.aruco.DetectorParameters()
detector = cv.aruco.ArucoDetector(aruco_dict, parameters)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detección de ArUco
    corners, ids, rejected = detector.detectMarkers(frame)

    if ids is not None:
        cv.aruco.drawDetectedMarkers(frame, corners, ids)

        for i, corner in enumerate(corners):
            c = corner[0]
            # Coordenadas del centro
            x_corner = int((c[:,0].mean()))
            y_center = int((c[:,1].mean()))
            cv.circle(frame, (x_corner, y_center), 5, (0,0,255), -1)

            print(f"Marker {ids[i]} en píxeles: ({x_corner},{y_center})")

    cv.imshow("Frame", frame)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

# Calibración de Cámara

Instrucciones de uso:

* Colocar en chess_size las dimensiones del tablero de ajedrez (es decir, la cantida de cuadrados en x e y)
* Colocar el tamaño de cada casilla en mm. Como son cuadradas por definición, solo se requiere el largo.


*  Se obtienen las variables: ret, mtx, dist, rvecs, tvecs

ret: Error de retención. (RMS de error general de la cámara).

mtx: Matriz de la cámara. (Matriz de la cámara. 3x3. Indica parámetros propios como el focal lenght).

dist: Coeficientes de distorción. **IMPORTANTE** (Array de 5 valores para corrección de la cámara. 3 de distorción radial y 2 de distorción tangencial).

rvecs: Vector de rotación. (Matriz de rotación de la imagen. Permite corregir la rotación).	**IRRELEVANTE**

tvecs: Vector de traslación. (Matriz de traslación de la imagen. Permite corregir la posición de la imagen).**IRRELEVANTE**


In [ ]:
import cv2 as cv
import numpy as np
import glob
import sys
import os

###### VARIABLES ###########################################################
chess_size = (9,6)					# Dimensiones de tablero patrón
chess_square_lenght = 30.0  		# En mm

calib_img_path = 'calibration_images'
test_img_path = 'test_images'

criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)




#########################################################################################################
# Vector de posiciones de esquinas:
corner_size = (chess_size[0]-1, chess_size[1]-1)    #Las funciones utilizan las esquinas internas, no la cant de cuadrados
objp = np.zeros((corner_size[0]*corner_size[1], 3), np.float32)
objp[:,:2] = np.mgrid[0:corner_size[0],0:corner_size[1]].T.reshape(-1,2)
objp *= chess_square_lenght

objpoints = [] # puntos 3D en el mundo real
imgpoints = [] # puntos 2D en la imagen

images = glob.glob(f'{calib_img_path}/*.jpg')

if not images:
    print("No hay imagenes.")
    sys.exit()
if len(images) < 10:
    print("Advertencia. Pocas imágenes. Se recomiendan 10 o más en distintos ángulos")

for fname in images:
    print(f"Probando foto {fname}")
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)		# Lo pasa a escala de grises para mejorar
    ret, corners = cv.findChessboardCornersSB(gray, corner_size, None)
    
    if ret == True:
        print(f"Encontrado en {fname}")
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)

        cv.drawChessboardCorners(img, corner_size, corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)

cv.destroyAllWindows()

if not ret:
    print("No se encontró nada.")
    sys.exit()

# Calibración
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

print("Matriz intrínseca:\n", mtx)
print("Coeficientes de distorsión:\n", dist)
np.savez("calibration_values", mtx=mtx, dist=dist)

# Verificacion
images = glob.glob(f'{test_img_path}/*.jpg')

if not images:
    print("No hay imagenes para verificar.")
    sys.exit()

for fname in images:
    print(f"Corrigiendo {fname}")
    img = cv.imread(fname)
    h,  w = img.shape[:2]
    newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

    # Elimina distorcion
    dst = cv.undistort(img, mtx, dist, None, newcameramtx)
 
    # Recorte de la imagen
    x, y, w, h = roi
    dst = dst[y:y+h, x:x+w]
    name = os.path.basename(fname)
    cv.imwrite(f"{test_img_path}/{name}_calib.jpg", dst)
    print(f"{fname} corregido y guardado en test_iamges/{name}_calib.jpg")

In [ ]:
import numpy as np

def calib_img(img, mtx, dist):
    h,  w = img.shape[:2]
    newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

    # Elimina distorcion
    undistorted = cv.undistort(img, mtx, dist, None, newcameramtx)
 
    # Recorte de la imagen
    x, y, w, h = roi
    undistorted = undistorted[y:y+h, x:x+w]
    return undistorted

def load_calib_matrix(file):
    try:
        calib = np.load(f"{file}.npz")
        mtx, dist = calib["mtx"], calib["dist"]
    except Exception as e:
        mtx, dist = None, None
    return mtx, dist

# Medición de distancia Aruco Marker

Prueba 1:


El marker size es 100mm pero no me devuelve el Z correcto. Me devuelve el doble. Con marker size de 50 funciona bien

In [ ]:
import cv2 as cv
import sys

###### VARIABLES ###########################################################
marker_size = 50  # mm

#######################################################################################
cap = cv.VideoCapture(0)

### Creación del diccionario y detector
aruco_dict = cv.aruco.getPredefinedDictionary(cv.aruco.DICT_4X4_100)
parameters = cv.aruco.DetectorParameters()
detector = cv.aruco.ArucoDetector(aruco_dict, parameters)

### Carga de la Matriz de calibracion
mtx, dist = load_calib_matrix("calibration_values")

if mtx is None or dist is None:
    print("Faltan valores de calibracion")
    sys.exit()


### Loop de lectura de video
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detección de ArUco. NOTA: La funcion detecta todos los ArUco en pantalla y los pasa como lista con los ID.
    # corners[i][0][corner_num] -> (xi, yi)
    # ids[i] -> id del marker i
    corners, ids, _ = detector.detectMarkers(frame)

    if ids is not None:
        # tvecs[i][0] -> (x,y,z)
        rvecs, tvecs, _ = cv.aruco.estimatePoseSingleMarkers(corners, marker_size, mtx, dist)

        cv.aruco.drawDetectedMarkers(frame, corners, ids)

        for i in range(len(ids)):
            cv.aruco.drawDetectedMarkers(frame, corners, ids)

            # Dibujar ejes
            cv.drawFrameAxes(frame, mtx, dist, rvecs[i], tvecs[i], 50)
            
            # Mostrar distancia Z (en mm)
            x, y, z = tvecs[i][0]

            print(f"marcador {ids[i]} en (x:{x:.2f},y:{x:.2f},z:{z:.2f})")

            x_corner = int(corners[i][0][3][0])
            y_corner = int(corners[i][0][3][1])
            
            
            #print(f"Marker {ids[i]} a {distancia:.1f} mm de la cámara")

            cv.putText(frame, f"Z: {z:.1f}",(x_corner, y_corner - 10),                       
                       cv.FONT_HERSHEY_SIMPLEX, 
                       0.8, 
                       (255, 0, 0), 
                       2, 
                       cv.LINE_AA)

    cv.imshow("Frame", frame)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()


Prueba 2: (El código fue sacado del siguiente [link](https://visionbrick.com/measuring-depth-from-a-single-camera-using-aruco-markers-in-opencv/))

In [ ]:
import cv2 as cv
import numpy as np

""" 
load your camera’s intrinsic matrix and distortion coefficients from a YAML file
- camera_matrix is the 3×3 intrinsic parameters matrix.
- dist_coeffs holds lens distortion parameters.
- marker_length is the side length of the ArUco marker in meters.
"""
camera_matrix, dist_coeffs = load_calib_matrix("calibration_values")
marker_length = 100  # mm
 
"""
DICT_4X4_50 is a predefined dictionary of ArUco markers.
4x4 markers with 50 unique IDs.(16 bits per marker)
parameters are the default detection parameters.
"""
aruco_dict = cv.aruco.getPredefinedDictionary(cv.aruco.DICT_4X4_100)
parameters = cv.aruco.DetectorParameters()
 
# create a detector instance with default parameters.
detector = cv.aruco.ArucoDetector(aruco_dict, parameters)
 
cap = cv.VideoCapture(0)
 
num=0
 
while True:
    ret, frame = cap.read()
    if not ret:
        break
 
    # Detect markers in the frame with detectMarkers method
    corners, ids, _ = detector.detectMarkers(frame)
    """
    Example output of corners:
    (array([[[ 15., 339.],
        [ 91., 334.],
        [ 98., 404.],
        [ 19., 414.]]], dtype=float32),)
    """
 
    if ids is not None and len(ids) > 0:
        cv.aruco.drawDetectedMarkers(frame, corners, ids)
 
        # Define the 3D coordinates of the marker corners in the marker's coordinate system
        obj_points = np.array([
            [-marker_length / 2,  marker_length / 2, 0],
            [ marker_length / 2,  marker_length / 2, 0],
            [ marker_length / 2, -marker_length / 2, 0],
            [-marker_length / 2, -marker_length / 2, 0]
        ], dtype=np.float32)
 
         
        for marker_corners in corners:
            image_points = marker_corners[0].astype(np.float32)
 
            """
            solvePnP estimates the pose of a 3D object given its 3D points and corresponding 2D image points.
            It returns the rotation vector (rvec) and translation vector (tvec).
            """
            retval, rvec, tvec = cv.solvePnP(obj_points, image_points, camera_matrix, dist_coeffs)
             
            if retval:
                # Draw the axis on the frame
                cv.drawFrameAxes(frame, camera_matrix, dist_coeffs, rvec, tvec, 0.03)
                 
                # Extract the translation vector and calculate the distance
                x, y, z = tvec.flatten()
                distance = np.linalg.norm(tvec)
                
                # Print to terminal
                print(f"X={x:.3f} m, Y={y:.3f} m, Z(depth)={z:.3f} m, Distance={distance:.3f} m")
                
                # Display on frame with different colors
                org = (20, 40)  
                font = cv.FONT_HERSHEY_SIMPLEX
                font_scale = 0.7
                thickness = 2
                cv.putText(frame, f"X={x:.3f} mm", (org[0], org[1]), font, font_scale, (0, 255, 0), thickness, cv.LINE_AA)
                cv.putText(frame, f"Y={y:.3f} mm", (org[0], org[1]+30), font, font_scale, (255, 0, 0), thickness, cv.LINE_AA)
                cv.putText(frame, f"Depth={z:.3f} mm", (org[0], org[1]+60), font, font_scale, (0, 0, 255), thickness, cv.LINE_AA)
                cv.putText(frame, f"Dist={distance:.3f} mm", (org[0], org[1]+90), font, font_scale, (0, 255, 255), thickness, cv.LINE_AA)
 
    # Display the frame with detected markers and pose estimatio
    cv.imshow('ArUco Pose Estimation', frame)
      
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
 
cap.release()
cv.destroyAllWindows()

# Rectificación de la foto

Utilizando 4 ArUco Markers se busca que el plano quede totalmente perpendicular. Este código debería ajustar eso y ver "desde arriba" al plano.

Se le conoce como **Homografía**.


Este código utiliza 4 ID's distintas. Una para cada esquina. Podría mejorarse a 1 sola cambiando:
```python:
    if marker_id in corner_ids:
```

Por:
```python:
    if marker_id == corner_id:
```

Y también cambiar:
```python:
    if all(mid in detected for mid in corner_ids)
```

Por:
```python:
    if (detected.len() == 4)
```

In [ ]:
import cv2 as cv
import numpy as np

## Variables:
calib_value_file = "calibration_values"

plane_size = (300, 200)     # En mm
image_size = (600, 600)     # Tamaño de la hoja En pixeles.

scale = plane_size / image_size

## Código:
mtx, dist = load_calib_matrix(calib_value_file)
cap = cv.VideoCapture(0)

aruco_dict = cv.aruco.Dictionary_get(cv.aruco.DICT_4X4_100)
parameters = cv.aruco.DetectorParameters_create()

corner_ids = [0, 1, 2, 3]   # Orden: 0=arriba izq, 1=arriba der, 2=abajo der, 3=abajo izq

while True:
    ret, frame = cap.read() # Solo para el caso de video. Omitir con fotos.
    if not ret:
        break

    # Corrección y detección de ArUco Marker:
    if mtx is not None and dist is not None:
        frame = calib_img(frame, mtx, dist)
    corners, ids, _ = cv.aruco.detectMarkers(frame, aruco_dict, parameters=parameters)

    if ids is not None:
        frame = cv.aruco.drawDetectedMarkers(frame, corners, ids)       # Solo visual

        # Busqueda de corners:
        ids = ids.flatten()
        detected = {}

        for i, marker_id in enumerate(ids):
            # Obtengo centros solo de los corners ID:
            if marker_id in corner_ids:
                c = corners[i][0]
                center_x, center_y = c[:,0].mean(), c[:,1].mean()
                detected[marker_id] = (center_x, center_y)
                cv.circle(frame, (int(center_x), int(center_y)), 5, (0,0,255), -1)  # Solo visual

        if all(mid in detected for mid in corner_ids):
            detected_points = np.array([
                detected[0],  # arriba izq
                detected[1],  # arriba der
                detected[2],  # abajo der
                detected[3]   # abajo izq
            ], dtype=np.float32)

            # Tamaño deseado de salida (ejemplo: 600x600 px)
            real_points = np.array([
                [0, 0],                             # arriba izq
                [image_size[0]-1, 0],               # arriba der
                [image_size[0]-1, image_size[1]-1], # abajo der
                [0, image_size[1]-1]                # abajo izq
            ], dtype=np.float32)

            # Corrección de la imagen: 
            H, _ = cv.findHomography(detected_points, real_points)
            aligned = cv.warpPerspective(frame, H, image_size)
            print(f"Imagen corregida con escala x:{scale[0]}, y:{scale[1]}")
            cv.imshow("Vista Alineada", aligned)

    cv.imshow("Camara", frame)

    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()